#Un encodage à chaud ou un encodage d'étiquettes à l'aide de Scikit-Learn

*Le codage catégoriel est le processus de conversion de colonnes catégorielles en colonnes numériques afin qu'un algorithme d'apprentissage automatique le comprenne. C'est un processus de conversion de catégories en nombres.*

**Différentes approches du codage catégoriel**

Alors, comment devrions-nous gérer les variables catégorielles ? Il s'avère qu'il existe plusieurs façons de gérer les variables catégorielles. Dans cet article, je vais aborder les deux techniques les plus utilisées :

***Encodage des étiquettes***

***Encodage à chaud***

**Qu’est-ce que l’encodage d’étiquettes ?**

*Le codage d'étiquettes est une technique de codage populaire pour gérer les variables catégorielles. Un nombre entier unique ou un ordre alphabétique représente chaque étiquette.*

Voyons comment implémenter le codage d'étiquettes en Python à l'aide de la bibliothèque scikit-learn et comprenons également les défis liés au codage d'étiquetes

In [ ]:
#Importez d’abord les bibliothèques et l’ensemble de données requis :

#importing the libraries
import pandas as pd
import numpy as np

#reading the dataset
data=pd.read_csv("/content/salary.csv")

In [ ]:
data

,Position,Level,Salary
0,Business Analyst,1,45000
1,Junior Consultant,2,50000
2,Senior Consultant,3,60000
3,Manager,4,80000
4,Country Manger,5,110000
5,Region Manager,6,150000
6,Partner,7,200000
7,Senior Partner,8,220000
8,C.level,9,250000
9,CEO,10,1000000


In [ ]:
#Comprendre les types de données des fonctionnalités :
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Position  10 non-null     object
 1   Level     10 non-null     int64 
 2   Salary    10 non-null     int64 
dtypes: int64(2), object(1)
memory usage: 368.0+ bytes


Comme vous pouvez le voir ici, la première colonne, Position, est la caractéristique catégorielle telle qu'elle est représentée par le type de données de l'objet  et les autres sont des caractéristiques numériques telles qu'elles sont représentées par int64 .

Maintenant, implémentons l'encodage des étiquettes en Python :

In [ ]:
# Import label encoder
from sklearn import preprocessing
# label_encoder object knows how to understand word labels.
label_encoder = preprocessing.LabelEncoder()
# Encode labels in column 'Country'.
data['Country']= label_encoder.fit_transform(data['Country'])
print(data.head())

Comme vous pouvez le voir ici, le codage des étiquettes utilise l'ordre alphabétique. Par conséquent, l’Inde a été codée avec 0, les États-Unis avec 2 et le Japon avec 1.

**Challenges with Label Encoding**

Dans le scénario ci-dessus, les noms de pays n’ont ni ordre ni rang. Mais lorsque le codage des étiquettes est effectué, les noms de pays sont classés en fonction des alphabets. Pour cette raison, il existe une très forte probabilité que le modèle capture la relation entre des pays tels que l'Inde < le Japon < les États-Unis.

**Qu’est-ce qu’un encodage à chaud ?**

*One-Hot Encoding est une autre technique populaire pour traiter les variables catégorielles. Il crée simplement des fonctionnalités supplémentaires basées sur le nombre de valeurs uniques dans la fonctionnalité catégorielle.* Chaque valeur unique de la catégorie sera ajoutée en tant que fonctionnalité. One-Hot Encoding est le processus de création de variables factices.

Dans cette technique de codage, chaque catégorie est représentée sous la forme d'un vecteur unique.

Voyons comment implémenter l'encodage one-hot en Python :

In [ ]:
# importing one hot encoder
from sklearn.preprocessing import OneHotEncoder
# creating one hot encoder object
onehotencoder = OneHotEncoder()
#reshape the 1-D country array to 2-D as fit_transform expects 2-D and finally fit the object
X = onehotencoder.fit_transform(data.Country.values.reshape(-1,1)).toarray()
#To add this back into the original dataframe
dfOneHot = pd.DataFrame(X, columns = ["Country_"+str(int(i)) for i in range(data.shape[1])])
df = pd.concat([data, dfOneHot], axis=1)
#droping the country column
df= df.drop(['Country'], axis=1)
#printing to verify
print(df.head())

Comme vous pouvez le voir ici, 3 nouvelles fonctionnalités sont ajoutées car le pays contient 3 valeurs uniques : l'Inde, le Japon et les États-Unis. Dans cette technique, nous avons résolu le problème du classement puisque chaque catégorie est représentée par un vecteur binaire.

**Défis de l'encodage One-Hot**: piège à variables factices

Le codage One-Hot entraîne un piège à variable factice, car le résultat d'une variable peut facilement être prédit à l'aide des variables restantes. Dummy Variable Trap est un scénario dans lequel les variables sont fortement corrélées les unes aux autres.

Le piège des variables factices conduit au problème connu sous le nom de multicolinéarité . La multicolinéarité se produit lorsqu'il existe une dépendance entre les entités indépendantes. La multicolinéarité est un problème sérieux dans les modèles d'apprentissage automatique tels que la régression linéaire et la régression logistique .

Ainsi, afin de surmonter le problème de multicolinéarité, l’une des variables muettes doit être supprimée. Ici, je vais démontrer pratiquement comment le problème de la multicolinéarité est introduit après avoir effectué le codage one-hot.

L'un des moyens courants de vérifier la multicolinéarité est le facteur d'inflation de variance (VIF) :


VIF=1, très moins de multicolinéarité

VIF <5, multicolinéarité modérée

VIF>5, Multicolinéarité Extrême (C'est ce qu'il faut éviter)

Calculez les scores VIF :

In [ ]:
# Function to calculate VIF
def calculate_vif(data):
    vif_df = pd.DataFrame(columns = ['Var', 'Vif'])
    x_var_names = data.columns
    for i in range(0, x_var_names.shape[0]):
        y = data[x_var_names[i]]
        x = data[x_var_names.drop([x_var_names[i]])]
        r_squared = sm.OLS(y,x).fit().rsquared
        vif = round(1/(1-r_squared),2)
        vif_df.loc[i] = [x_var_names[i], vif]
    return vif_df.sort_values(by = 'Vif', axis = 0, ascending=False, inplace=False)

X=df.drop(['Salary'],axis=1)
calculate_vif(X)

À partir du résultat, nous pouvons voir que les variables factices créées à l'aide d'un codage à chaud ont un VIF supérieur à 5. Nous avons un problème de multicolinéarité.

Maintenant, supprimons l'une des variables factices pour résoudre le problème de multicolinéarité :

À partir du résultat, nous pouvons voir que les variables factices créées à l'aide d'un codage à chaud ont un VIF supérieur à 5. Nous avons un problème de multicolinéarité.

Maintenant, supprimons l'une des variables factices pour résoudre le problème de multicolinéarité :

In [ ]:
df = df.drop(df.columns[[0]], axis=1)
calculate_vif(df)

Ouah! Le VIF a diminué. Nous avons résolu le problème de la multicolinéarité. L’ensemble de données est désormais prêt pour la création du modèle.

**Encodage One-Hot vs encodage d'étiquettes**

***Méthode	Encodage à chaud	Encodage des étiquettes***

Description	Convertit chaque valeur catégorielle en vecteur binaire, créant ainsi de nouvelles colonnes binaires pour chaque catégorie.

Exemple	Catégorie originale : « Rouge »

Codé : [1, 0, 0]

Exemple	Catégorie originale : « Vert »

Codé : [0, 0, 1]

Exemple	Catégorie originale : « Bleu »


Codé : [1, 0, 0]

***Encodage des étiquettes***

Attribue une étiquette numérique unique à chaque catégorie, en préservant la relation ordinale si elle est présente.

Catégorie originale : « Rouge »

Codé : 1

Catégorie originale : « Vert »

Codé : 2

Catégorie originale : « Bleu »

Codé : 3

**Quand utiliser un encodage d’étiquettes ou un encodage à chaud ?**

Cette question dépend généralement de votre ensemble de données et du modèle que vous souhaitez appliquer. Mais quand même, quelques points à noter avant de choisir la bonne technique d’encodage pour votre modèle :

***Nous appliquons le One-Hot Encoding lorsque :***

1.   La caractéristique catégorielle n'est pas ordinale (comme les pays ci-dessus)
2.   Le nombre de fonctionnalités catégorielles est moindre, ce qui permet d'appliquer efficacement l'encodage à chaud.

***Nous appliquons le codage d'étiquette lorsque :***

1. La caractéristique catégorielle est ordinale (comme Jr. kg, Sr. kg, école
primaire, lycée)
2.  Le nombre de catégories est assez important car l'encodage à chaud peut entraîner une consommation de mémoire élevée.

**Encodage d'étiquettes, encodage à chaud ou encodage ordinal**

***Encodage d'étiquette :***  l'encodage d'étiquette attribue une étiquette numérique unique à chaque catégorie dans une variable catégorielle. Il préserve la relation ordinale entre les catégories si elle est présente. Par exemple, « Rouge » peut être codé comme 1, « Vert » comme 2 et « Bleu » comme 3.

***Encodage One-Hot:***  l'encodage One-Hot convertit chaque catégorie d'une variable catégorielle en un vecteur binaire. Il crée de nouvelles colonnes binaires pour chaque catégorie, représentant la présence ou l'absence de la catégorie. Chaque catégorie s'exclut mutuellement. Par exemple, « Rouge » peut être codé comme [1, 0, 0], « Vert » comme [0, 1, 0] et « Bleu » comme [0, 0, 1].

**Encodage ordinal:** l'encodage ordinal est similaire à l'encodage d'étiquettes mais prend en compte l'ordre ou le rang des catégories. Il attribue des étiquettes numériques uniques à chaque catégorie, préservant la relation ordinale entre les catégories. Par exemple, « Cold » peut être codé comme 1, « Warm » comme 2 et « Hot » comme 3.